In [ ]:
import time

import datetime

import math

import sys

import numpy as np

import pandas as pd



# binance - python 관련 import

import binance

from binance.client import Client

from binance.enums import *



# 이메일 발송을 위한 import

import smtplib

from email.mime.text import MIMEText

from email.mime.multipart import MIMEMultipart

from email.mime.base import MIMEBase

from email import encoders



# 바이낸스 API Key, Password

api_key = '여기에 API Key를 넣으세요.'

secret = '여기에 API Password를 넣으세요'



# 현재 실물계좌의 잔고를 계산하는 함수

def getCurrentNetValue(dic_balance_free, df_Tickers):

    sum = 0.0

    for key, value in dic_balance_free.items():

        if (key == 'BTC'):

            sum += value

        else:

            if (key + 'BTC') in df_Tickers.index:

                sum += float(df_Tickers.loc[key + 'BTC', 'lastPrice']) * float(value)

        

    return sum



# 이메일 발송하는 클래스

class sendMail:

    def send(self, strTitle, strContents):

        try:

            toEmail = '정보를 받을 이메일 주소'

            fromEmail = '정보를 보낼 이메일 주소'



            msg = MIMEMultipart()

            msg['From'] = fromEmail

            msg['To'] = toEmail

            msg['Subject'] = strTitle



            msg.attach(MIMEText(strContents,'plain'))



            text = msg.as_string()

            server = smtplib.SMTP_SSL('smtp.daum.net',465)        # 보내는 메일이 Daum 메일일 경우 세팅

            server.login('정보를 보낼 이메일 아이디', '비밀번호')



            server.sendmail(fromEmail, toEmail, text)

            server.quit()



            return 'Success to send emails.'



        except Exception as e:

            return "Failed. error" + str(e)



# 해당 종목을 팔아야 하는지 판단하는 함수

def IsSell(symbol):

    global client

    global dict_Ticker_Info

    global df_Tickers



    # 기초자료: Candle

    # 5분 캔들 자료를 받아와서 사용

    try:

        candles = client.get_klines(symbol=symbol, interval=Client.KLINE_INTERVAL_5MINUTE)

    except:        

        print("Unexpected error:", sys.exc_info()[0])

        return False



    # 5분 캔들을 사용하기 편한 DataFrame 형태로 변경

    columns =['OpenTime', 'Open', 'High', 'Low', 'Close', 'Volume', 'CloseTime', 'QuoteAssetVolume', \

    'NumberOfTrades', 'TakerBuyBaseAssetVolume' , 'TakerBuyQuoteAssetVolume', 'Ignore']



    df_candles = pd.DataFrame(candles, columns = columns)[['Open', 'High', 'Low', 'Close', 'Volume']].astype(float) * 1000

    df_candles['Volume'] = df_candles['Volume'] / 1000.0

    lastIndex = len(df_candles) - 1

    

    # 샀을 때 가격 기준으로 팔지 결정

    price = dict_Ticker_Info[symbol][0]

    if (symbol in dict_Ticker_Info.keys()):

        # 샀을 때보다 1.5% 이상 올랐으면 판다

        if ((price - dict_Ticker_Info[symbol][0]) / dict_Ticker_Info[symbol][0]) >= 0.015:

            return True

        # 아몰랑 2% 손절

        elif ((dict_Ticker_Info[symbol][0] - price) / dict_Ticker_Info[symbol][0]) >= 0.015:

            return True



    # 고가와 저가가 붙어있는 이상한 케이스를 제외

    if (df_candles.loc[lastIndex,'High'] - df_candles.loc[lastIndex,'Low']) == 0.0:

        return False



    # 고가와 저가 사이에서 종가가 고가쪽에 붙어 있으면 어깨쯤이라고 판단해서 판다

    if ((df_candles.loc[lastIndex,'High'] - df_candles.loc[lastIndex,'Close']) / (df_candles.loc[lastIndex,'High'] - df_candles.loc[lastIndex,'Low'])) <= 0.25:

        return True



    return False



# 종목을 사야하는지 판단하는 함수

def IsBuy(symbol, price):

    global client



    # 기초자료: Candle

    try:

        candles = client.get_klines(symbol=symbol, interval=Client.KLINE_INTERVAL_5MINUTE)

    except:

        print("Unexpected error:", sys.exc_info()[0])

        return False



    columns =['OpenTime', 'Open', 'High', 'Low', 'Close', 'Volume', 'CloseTime', 'QuoteAssetVolume', \

    'NumberOfTrades', 'TakerBuyBaseAssetVolume' , 'TakerBuyQuoteAssetVolume', 'Ignore']



    df_candles = pd.DataFrame(candles, columns = columns)[['Open', 'High', 'Low', 'Close', 'Volume']].astype(float) * 1000

    df_candles['Volume'] = df_candles['Volume'] / 1000.0

    lastIndex = len(df_candles) - 1



    # Candle

    if (df_candles.loc[lastIndex,'High'] - df_candles.loc[lastIndex,'Low']) == 0.0:

        return False



    # 고가와 저가 사이에서 종가가 저가쪽에 붙어 있으면 무릎쯤이라고 판단해서 산다

    if ((price - df_candles.loc[lastIndex,'Low']) / (df_candles.loc[lastIndex,'High'] - df_candles.loc[lastIndex,'Low'])) < 0.15:

        return True

    

    return False



# binance client 접속

client = Client(api_key, secret)



# 프로그램 시작 시 시간, 지갑 전체 값

init_date = time.time()

init_NetValue = -1.0



# 이메일 발송 클래스 객체

objMail = sendMail()



# 10분 마다 이메일을 발송하기 위한 타임 플래스

refresh_date = init_date



# 잔고의 최초 매수가, 잔량을 기록하는 dict를 만든다. (정보 dict)

dict_Ticker_Info = dict()



# 최소 요청가능한 수량 (BTC 기준)

# df_exchange_info : 최소 요청가능한 수량 (BTC 기준) DataFrame

# 바이낸스에는 최소 거래량이 있어서 이 거래량보다 작은 양의 거래는 오류가 발생

exchange_info = client.get_exchange_info()['symbols']

df_exchange_info = pd.DataFrame.from_dict(exchange_info).set_index('symbol')

time.sleep(1)



# 바이낸스와 통신하는 부분에 1초씩 대기 시간이 들어가는 이유는 API요청이 너무 잦으면 계정 블록 당함.

while True:

    # 잔고를 점검한다.

    # dic_balance_free : 각 코인 별 가용수량 dict

    balances = client.get_account()['balances']

    balances_free = list(filter(lambda x: float(x['free']) > 0.0 , balances))

    dic_balance_free = { k['asset']: float(k['free']) for k in balances_free }

    time.sleep(1)



    # 시장정보

    # df_Tickers : 각 코인 간 시장정보 DataFrame

    # df_Tickers_By_Key : df_Tickers 을 거래횟수(count) 순서로 내림차순 정렬

    tickers = client.get_ticker()

    df_Tickers = pd.DataFrame(tickers).set_index('symbol').astype(float)

    time.sleep(1)



    # 초기 지갑 가치가 없으면 현재 가치를 기록

    if (init_NetValue == -1.0):

        init_NetValue = getCurrentNetValue(dic_balance_free, df_Tickers)

        objMail.send('작업시작','작업을 시작했습니다.\r\n현재가치:%f' % init_NetValue)

        refresh_date = datetime.datetime.now()



    # 현재 지갑 가치

    current_NetValue = getCurrentNetValue(dic_balance_free, df_Tickers)

    if (datetime.datetime.now() - refresh_date) >= datetime.timedelta(minutes=10):

        refresh_date = datetime.datetime.now()

        objMail.send('Alive - 초기가치:%f, 현재가치:%f, 수익률:%f' % (init_NetValue, current_NetValue, (current_NetValue/init_NetValue - 1.0)),'Alive')



    # 마진거래에서 사용되는 모든 BTC 기반 symbol

    info = client.get_all_isolated_margin_symbols()

    all_Isolated_Margin_Symbol = list(filter(lambda x: x['quote'] == 'BTC' , info))

    # symbol   base quote  isMarginTrade  isBuyAllowed  isSellAllowed

    df_All_Isolated_Margin_Symbol = pd.DataFrame.from_dict(all_Isolated_Margin_Symbol)

    time.sleep(1)



    # 현물지갑의 BTC 잔고

    # 'makerCommission', 'takerCommission', 'buyerCommission', 'sellerCommission', 'canTrade', 'canWithdraw',

    # 'canDeposit', 'updateTime', 'accountType', 'balances', 'permissions'

    account = client.get_account()

    balances_BTC = float(list(filter(lambda x: x['asset'] == 'BTC' , account['balances']))[0]['free'])

    time.sleep(1)



    # 거래횟수 내림차순 정리

    df_Tickers = pd.DataFrame(client.get_ticker()).set_index('symbol').astype(float)

    df_Tickers_By_Count = df_Tickers.loc[:, ['highPrice', 'bidPrice', 'askPrice', 'lowPrice', 'lastPrice', 'count']]

    df_Tickers_By_Count.index = df_Tickers_By_Count.index + '*'

    df_Tickers_By_Count = df_Tickers_By_Count.filter(like='BTC*', axis=0)

    df_Tickers_By_Count.index = df_Tickers_By_Count.index.str.replace('BTC*', 'BTC', regex=False)

    df_Tickers_By_Count = df_Tickers_By_Count.sort_values(by='count', ascending=False)

    # 거래 건수가 20000건 이하인 종목은 제외한다.

    df_Tickers_By_Count = df_Tickers_By_Count.loc[df_Tickers_By_Count.loc[:,'count'] > 20000, :]

    time.sleep(1)



    # 현물계좌에 BTC 잔고가 있다면 매수한다.

    if (balances_BTC > 0.0):

        for symbol in df_Tickers_By_Count.index:

            if (len(df_All_Isolated_Margin_Symbol.loc[(df_All_Isolated_Margin_Symbol.loc[:,'symbol'] == symbol),'symbol']) == 0):

                continue



            if (IsBuy(symbol, df_Tickers.loc[symbol,'bidPrice']) == False):

                continue



            # 전 금액을 고립마진에 넣는다.

            try:

                transaction = client.transfer_spot_to_isolated_margin(asset='BTC', symbol=symbol, amount=balances_BTC)

            except:

                continue

            

            time.sleep(1)



            # 금액 전송이 완료될 때까지 대기

            while True:

                info = client.get_isolated_margin_account(symbols=symbol)

                if (float(info['assets'][0]['quoteAsset']['free']) > 0):

                    time.sleep(1)

                    break

                else:

                    print('gap: %f' % (balances_BTC - float(info['assets'][0]['quoteAsset']['free'])))

                    time.sleep(1)

                    continue



            # 최대 대출 가능 금액

            details = client.get_max_margin_loan(asset='BTC', isolatedSymbol=symbol)

            avail_Amount = float(details['amount'])

            time.sleep(1)



            # 요청량

            tickers = client.get_ticker()

            df_Tickers = pd.DataFrame(tickers).set_index('symbol').astype(float)

            length = df_exchange_info.loc[symbol,['filters']]['filters'][0]['minPrice'].find('1') - 1

            price = round(df_Tickers.loc[symbol,'bidPrice'] * 1.0003, length)



            stepSize = float(df_exchange_info.filters[symbol][2]['stepSize'])

            amount_checked = round(math.floor((avail_Amount + balances_BTC) / price * 0.98 / stepSize) * stepSize, int(-1 * math.log10(stepSize)))



            # 전부 구매

            try:

                order = client.create_margin_order(

                    symbol=symbol,                  # 거래하고자 하는 종목

                    isIsolated='TRUE',              # 고립마진 거래 (고립, 교차)

                    side=SIDE_BUY,                  # 매수

                    type=ORDER_TYPE_LIMIT,          # 지정가 매매

                    sideEffectType="MARGIN_BUY",    # 구매할 때 자동으로 최대 대출

                    timeInForce=TIME_IN_FORCE_GTC,  # Timestamp 관련

                    price=price,                    # 지정가

                    quantity=amount_checked         # 매매량

                    )

                time.sleep(1)



                while True:

                    result = client.get_all_margin_orders(symbol=symbol, isIsolated='TRUE')

                    if result[-1]['status'] == 'FILLED':

                        break

                    else:

                        time.sleep(1)



                dict_Ticker_Info[symbol] = [price, amount_checked]

                

                objMail.send('매수알림','%s: %f에 %f만큼 매수했습니다.' % (symbol, price, amount_checked))



                time.sleep(2)

                break

            except:

                #print("Unexpected error:", sys.exc_info()[1].message)

                time.sleep(1)

                break



    # 가지고 있는 마진 중에 가격이 오른 것은 판다.

    isolate_Margin_Account = client.get_isolated_margin_account()

    time.sleep(1)



    isolate_Margin_Account_Asset = list(filter(lambda x: float(x['baseAsset']['free']) > 0 , isolate_Margin_Account['assets']))



    for item in isolate_Margin_Account_Asset:

        if item['symbol'] not in dict_Ticker_Info.keys():

            continue



        if (IsSell(symbol) == True):

            amount = dict_Ticker_Info[symbol][1]



            tickers = client.get_ticker()

            df_Tickers = pd.DataFrame(tickers).set_index('symbol').astype(float)

            length = df_exchange_info.loc[symbol,['filters']]['filters'][0]['minPrice'].find('1') - 1

            price = round(df_Tickers.loc[symbol,'askPrice'] * 0.9997, length)



            # 최소 거래 단위

            stepSize = float(df_exchange_info.filters[symbol][2]['stepSize'])

            # 최소 거래 단위가 반영된 거래량

            amount_checked = round(math.floor(amount * 0.98 / stepSize) * stepSize, int(-1 * math.log10(stepSize)))



            min_notional = float(df_exchange_info.filters[symbol][3]['minNotional']) / price

            if min_notional > amount_checked:

                time.sleep(1)

                continue

            

            time.sleep(1)

            try:

                order = client.create_margin_order(

                    symbol=symbol,

                    isIsolated='TRUE',

                    side=SIDE_SELL,                 # 매수

                    type=ORDER_TYPE_MARKET,         # 시장가 거래

                    sideEffectType="AUTO_REPAY",    # 대출금 자동 상환

                    quantity=amount_checked

                    )

                

                objMail.send('매도알림','%s: %f에 %f만큼 매도했습니다.\r\n수익률:%f'

                % (symbol, float(order['fills'][len(order['fills']) - 1]['price']), float(order['executedQty']),

                ((price - dict_Ticker_Info[symbol][0]) / dict_Ticker_Info[symbol][0])))



                del(dict_Ticker_Info[symbol])

                time.sleep(1.5)

                break

            except:

                print("Unexpected error:", sys.exc_info()[1].message)

                time.sleep(1)

                break



    #   남아 있는 free 자산들 모두 현물지갑으로 이동

    isolate_Margin_Account = client.get_isolated_margin_account()

    time.sleep(1)

    isolate_Margin_Account_Free = list(filter(lambda x: float(x['quoteAsset']['free']) > 0 , isolate_Margin_Account['assets']))

    

    for item in isolate_Margin_Account_Free:

        try:

            transaction = client.transfer_isolated_margin_to_spot(asset='BTC', symbol=item['symbol'], \

             amount=item['quoteAsset']['free'])

        except:

            time.sleep(1)

            continue

            

        time.sleep(1)



    time.sleep(1)